<a href="https://colab.research.google.com/github/Tobon74/Reto-Life-Expectancy-Data/blob/main/Reto_Life_Expectancy_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [5]:
# Cargar el archivo nuevamente con separador de punto y coma
file_path = "/content/Reto Life Expectancy Data(Tabla Original).csv"
df = pd.read_csv(file_path, sep=';')

In [6]:
# Limpiar nombres de columnas
df.columns = df.columns.str.strip()


In [7]:

# Eliminar filas con instrucciones escritas (no datos válidos)
df = df[df['Life expectancy'].str.replace(',', '.', regex=False).str.match(r'^\d+(\.\d+)?$')]

In [9]:
# Convertir columnas numéricas que estaban con comas a punto decimal
numeric_columns = [
    'Adult Mortality', 'infant deaths', 'Alcohol',
    'percentage expenditure', 'Hepatitis B', 'Measles', 'BMI'
]

for col in numeric_columns:
    # Check if the column is of object type before applying .str accessor
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

display(df.head())

,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,Year,"=COEF.DE.CORREL(B2:B2721,C2:C2721)",Unnamed: 11
0,Developing,65.0,263,62,0.01,71.279624,65.0,1154,19.1,2015,"=COEF.DE.CORREL(D2:D2721,E2:E2721)",Instrucciones
1,Developing,59.9,271,64,0.01,73.523582,62.0,492,18.6,2014,"=COEF.DE.CORREL(F2:F2721,G2:G2721)",El reto es generar un análisis exploratorio co...
2,Developing,59.9,268,66,0.01,73.219243,64.0,430,18.1,2013,"=COEF.DE.CORREL(H2:H2721,I2:I2721)",NaN
3,Developing,59.5,272,69,0.01,78.184215,67.0,2787,17.6,2012,NaN,1) Tener un Análisis de completitud
4,Developing,59.2,275,71,0.01,7.097109,68.0,3013,17.2,2011,NaN,2) Identificar variables cualitativas y cuanti...


In [10]:
# Convertir otras columnas si aplican
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [11]:
# Análisis de completitud
missing_data = df[numeric_columns + ['Status']].isnull().sum().reset_index()
missing_data.columns = ['Variable', 'Valores Faltantes']

In [13]:
# Identificación de tipos de variable
var_types = pd.DataFrame({
    'Variable': df.columns,
    'Tipo': df.dtypes.map(lambda x: 'Cuantitativa' if x in ['float64', 'int64'] else 'Cualitativa')
}) # Added the missing closing parenthesis
display(var_types) # Added display to show the result

,Variable,Tipo
Status,Status,Cualitativa
Life expectancy,Life expectancy,Cuantitativa
Adult Mortality,Adult Mortality,Cuantitativa
infant deaths,infant deaths,Cuantitativa
Alcohol,Alcohol,Cuantitativa
percentage expenditure,percentage expenditure,Cuantitativa
Hepatitis B,Hepatitis B,Cuantitativa
Measles,Measles,Cuantitativa
BMI,BMI,Cuantitativa
Year,Year,Cuantitativa


In [14]:
# Matriz de correlación
correlation_matrix = df[numeric_columns].corr()

In [15]:
# Regresión lineal múltiple
from sklearn.linear_model import LinearRegression

X = df[['Alcohol', 'BMI', 'infant deaths', 'percentage expenditure', 'Adult Mortality']].dropna()
y = df.loc[X.index, 'Life expectancy']

model = LinearRegression()
model.fit(X, y)
regression_results = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': model.coef_
})
regression_results.loc[len(regression_results)] = ['Intercepto', model.intercept_]


In [16]:
# Crear hoja de resumen
resumen = [
    ["Apunte de Investigación – Análisis Exploratorio"],
    [""],
    ["1. Análisis de Completitud:"],
    ["Se identificaron valores faltantes en variables numéricas, que fueron tratados mediante conversión a tipo numérico y eliminación de registros inválidos."],
    [""],
    ["2. Variables Cualitativas y Cuantitativas:"],
    ["Se clasificaron las variables en cualitativas (ej. 'Status') y cuantitativas (Life expectancy, Alcohol, etc.)."],
    [""],
    ["3. Matriz de Correlación:"],
    ["Se generó una matriz de correlación entre variables numéricas para detectar relaciones lineales."],
    [""],
    ["4. Regresión Lineal Múltiple:"],
    ["Se construyó un modelo de regresión para predecir la esperanza de vida."],
    ["Las variables más influyentes fueron: Alcohol, BMI y Adult Mortality."],
    [""],
    ["Observaciones técnicas:"],
    ["- Se corrigieron comas decimales por puntos."],
    ["- Se eliminaron filas con texto o encabezados dentro del cuerpo de datos."],
    ["- Se manejaron valores faltantes usando dropna en la regresión."]
]

In [19]:
!pip install xlsxwriter

In [21]:
# Guardar todas las hojas en un nuevo archivo Excel
output_path = "/content/Analisis_Exploratorio_Completo.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name="Datos Limpios")
    missing_data.to_excel(writer, index=False, sheet_name="Completitud")
    var_types.to_excel(writer, index=False, sheet_name="Tipos de Variables")
    correlation_matrix.to_excel(writer, sheet_name="Matriz Correlación")
    regression_results.to_excel(writer, index=False, sheet_name="Regresión Lineal")

In [23]:
# Hoja resumen
resumen_df = pd.DataFrame(resumen)
resumen_df.to_excel(writer, index=False, header=False, sheet_name="Informe")

output_path

'/content/Analisis_Exploratorio_Completo.xlsx'